# Product Reccomendation using A-PRIORI algorithm
##### Aniruddha Dutta   1968324

Data Used: browsing.txt

Code: This code was prepared in google colab

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 41.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=b8e1c96e379bc49675019c5179f935240526e53baf9ec904a02435309e8e65f8
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from tqdm import tqdm

In [0]:
# create the session
#conf = SparkConf().set("spark.ui.port", "4050")
conf = SparkConf().set('spark.executor.memory', '10g')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-17 04:27:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.85.243.28, 3.220.9.69, 3.229.178.204, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.85.243.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  85.1MB/s    in 0.2s    

2020-04-17 04:27:04 (85.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.6/json/

In [0]:
#loading data. I uploaded the txt file in colab and read it from there directly
browsing = sc.textFile("browsing.txt")
browsing.take(3)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ']

In [0]:
browsing_1 = browsing.map(lambda x:(x, 1))
browsing_1.take(3)

[('FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ', 1),
 ('GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
  1),
 ('ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ', 1)]

####**First pass:** to get most frequent singletons with support atleast 100.
I am trying to get one productIDs in a single row to be able to count their number of occurances for calculation of support.

In [0]:
c1_df = spark.createDataFrame((browsing_1), ["products","count"])
c1_df.head(5)

[Row(products='FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ', count=1),
 Row(products='GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ', count=1),
 Row(products='ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ', count=1),
 Row(products='ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ', count=1),
 Row(products='ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ', count=1)]

In [0]:
c1 = [str(row.products) for row in  c1_df.select('products').collect()]
c1 = sc.parallelize(c1).flatMap(lambda x: x.split(' '))
c1.take(3)

['FRO11987', 'ELE17451', 'ELE89019']

In [0]:
c1_rdd = c1.map(lambda x:(x,1))
c1_rdd.take(10)

[('FRO11987', 1),
 ('ELE17451', 1),
 ('ELE89019', 1),
 ('SNA90258', 1),
 ('GRO99222', 1),
 ('', 1),
 ('GRO99222', 1),
 ('GRO12298', 1),
 ('FRO12685', 1),
 ('ELE91550', 1)]

In [0]:
#adding all the 1s grouped by the corresponding productID to get their frequency
t = c1_rdd.reduceByKey(lambda a, b: a+b).sortBy(lambda r: -r[1])
t.take(5)

[('', 31101),
 ('DAI62779', 6667),
 ('FRO40251', 3881),
 ('ELE17451', 3875),
 ('GRO73461', 3602)]

In [0]:
#Sanity check: Number of frequent singletons should be 647
L1 = spark.createDataFrame((t), ["ProductId","Frequency"])
L1_singletons = L1.filter(L1.Frequency >= 100).filter(L1.ProductId != '')
L1_singletons.count()

647

####**Second Pass:** Using the list of most frequent productIDs from the first pass, create pairs of productIDs and find the most frequent occuring pairs.

In [0]:
#2nd pass
#Crossjoined the singletons table to create productID pairs
L1_singletons.registerTempTable('singletons')

query = """SELECT s1.ProductId as p1, s2.ProductId as p2
            FROM singletons s1 cross join singletons s2"""
L1_crossjoined = spark.sql(query)
L1_crossjoined.head(3)

[Row(p1='DAI62779', p2='DAI62779'),
 Row(p1='DAI62779', p2='FRO40251'),
 Row(p1='DAI62779', p2='ELE17451')]

In [0]:
#This function will iterate over every row of the crossjoined rdd to combine the seperate productIDs into one string
def doubles(row):
  p1 = row['p1']
  p2 = row['p2']
  basket_2 = p1 + ' ' + p2
  return (basket_2,1)


In [0]:
c2 = L1_crossjoined.rdd.map(lambda row: doubles(row))
c2

PythonRDD[59] at RDD at PythonRDD.scala:53

In [0]:
c2.take(3)

[('DAI62779 DAI62779', 1), ('DAI62779 FRO40251', 1), ('DAI62779 ELE17451', 1)]

In [0]:
c2_df = spark.createDataFrame((c2), ["product_pair","count"])

In [0]:
c1_df.registerTempTable("browsing")
c2_df.registerTempTable("pairs")

In [0]:
#This function is used to calculate the support for {X,Y} by matching them in the browsing history data
def second_pass(row):
  pair = row['product_pair']
  p1 = row['product_pair'].split(" ")[0]
  p2 = row['product_pair'].split(" ")[1]
  browsing_len = len(browsing_history)
  count = 0

  if(p1 != p2):
    for i in range(browsing_len):
      if((p1 in browsing_history[i]) and (p2 in browsing_history[i])):
        count = count+1
  else:
    count = 0

  

  return (pair, int(count))

In [0]:
browsing_history = browsing.map(lambda row: ((row))).collect()
browsing_history[1]

'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 '

In [0]:
l2 = c2_df.rdd.map(lambda row: second_pass(row))
l2.take(3)

[('DAI62779 DAI62779', 0),
 ('DAI62779 FRO40251', 1070),
 ('DAI62779 ELE17451', 1592)]

In [0]:
def funct(row):
  pid_x = row[0].split(" ")[0]
  pid_y = row[0].split(" ")[1]
  s_xy = row[1]
  return (pid_x, pid_y, int(s_xy))

In [0]:
l2_rdd = l2.map(lambda row:funct(row))
l2_rdd.take(2)

[('DAI62779', 'DAI62779', 0), ('DAI62779', 'FRO40251', 1070)]

In [0]:
#Keeping only those pairs whose suppport is >= 100
l2_rdd_1 = l2_rdd.filter(lambda x:x[2]>=100) 
l2_withconf = spark.createDataFrame((l2_rdd_1), ["X", "Y", "S_XY"])
l2_withconf.show(5)

+--------+--------+----+
|       X|       Y|S_XY|
+--------+--------+----+
|DAI62779|FRO40251|1070|
|DAI62779|ELE17451|1592|
|DAI62779|GRO73461|1139|
|DAI62779|SNA80324| 923|
|DAI62779|ELE32164| 832|
+--------+--------+----+
only showing top 5 rows



Calculate the confidence measure for the most frequesnt association rules

In [0]:

l2_withconf.registerTempTable("l2_rdd")

query = """select l22.*, s2.ProductId as Y_1,s2.Frequency as S_Y 
            from(
            select l2.X as X, l2.Y as Y, l2.S_XY as S_XY, s1.ProductId as X_1, s1.Frequency as S_X
            from l2_rdd l2  left join singletons s1
            on l2.X = s1.ProductId) as l22
            left join singletons s2
            on l22.Y = s2.ProductId"""
l2_forconfcalculation = spark.sql(query)
l2_forconfcalculation.show(3)

+--------+--------+----+--------+----+--------+----+
|       X|       Y|S_XY|     X_1| S_X|     Y_1| S_Y|
+--------+--------+----+--------+----+--------+----+
|GRO71621|SNA96271| 105|GRO71621|1920|SNA96271|1295|
|FRO40251|SNA96271| 227|FRO40251|3881|SNA96271|1295|
|GRO38983|SNA96271| 102|GRO38983| 839|SNA96271|1295|
+--------+--------+----+--------+----+--------+----+
only showing top 3 rows



In [0]:
def conf_calc_xy(row):
  pid_x = row['X']
  pid_y = row['Y']
  conf_xy = float(row['S_XY'])/float(row['S_X'])
  xy = pid_x + "->" + pid_y
  return (pid_x, pid_y, xy, float(conf_xy))

In [0]:
def conf_calc_yx(row):
  pid_x = row['X']
  pid_y = row['Y']
  conf_yx = float(row['S_XY'])/float(row['S_Y'])
  yx = pid_y + "->" + pid_x
  return (pid_y, pid_x, yx, float(conf_yx))

In [0]:
#Calculate confidence for {X -> Y} and {Y->X} rules
l2_results_xy = l2_forconfcalculation.rdd.map(lambda row: conf_calc_xy(row))
l2_results_yx = l2_forconfcalculation.rdd.map(lambda row: conf_calc_yx(row))

In [0]:
l2_results = l2_results_xy.union(l2_results_yx)
l2_results.take(5)

[('GRO71621', 'SNA96271', 'GRO71621->SNA96271', 0.0546875),
 ('FRO40251', 'SNA96271', 'FRO40251->SNA96271', 0.05849007987632054),
 ('GRO38983', 'SNA96271', 'GRO38983->SNA96271', 0.12157330154946365),
 ('ELE32164', 'SNA96271', 'ELE32164->SNA96271', 0.035426166257453526),
 ('GRO73461', 'SNA96271', 'GRO73461->SNA96271', 0.06246529705719045)]

In [0]:
L2_rules = l2_results.sortBy(lambda r: -r[3])
L2_rules.take(5)
#The data has duplicates

[('DAI93865', 'FRO40251', 'DAI93865->FRO40251', 1.0),
 ('DAI93865', 'FRO40251', 'DAI93865->FRO40251', 1.0),
 ('GRO85051', 'FRO40251', 'GRO85051->FRO40251', 0.999176276771005),
 ('GRO85051', 'FRO40251', 'GRO85051->FRO40251', 0.999176276771005),
 ('GRO38636', 'FRO40251', 'GRO38636->FRO40251', 0.9906542056074766)]

In [0]:
L2_rules = L2_rules.distinct() #Removing duplicates
L2_rules.sortBy(lambda r: -r[3]).take(5)

#Sanity Check: all the top-5 associatio rules have confidence score > 0.985

[('DAI93865', 'FRO40251', 'DAI93865->FRO40251', 1.0),
 ('GRO85051', 'FRO40251', 'GRO85051->FRO40251', 0.999176276771005),
 ('GRO38636', 'FRO40251', 'GRO38636->FRO40251', 0.9906542056074766),
 ('ELE12951', 'FRO40251', 'ELE12951->FRO40251', 0.9905660377358491),
 ('DAI88079', 'FRO40251', 'DAI88079->FRO40251', 0.9867256637168141)]

####**Third Pass:** Using the most frequent doubles, form triplets and identify the most frequent itemset

In [0]:
#Make a list of productIDs from the second_pass results
p_list_1 = L2_rules.map(lambda x:x[0]).collect()
p_list_2 = L2_rules.map(lambda x:x[1]).collect()
p_list = p_list_1 + p_list_2

In [0]:
p_list = list(dict.fromkeys(p_list))
len(p_list)

222

In [0]:
p_list = sc.parallelize(p_list).map(lambda x:(x,1))
p_list

PythonRDD[146] at RDD at PythonRDD.scala:53

In [0]:
p_list.take(2)

[('SNA40784', 1), ('ELE91337', 1)]

In [0]:
p_list_df = spark.createDataFrame((p_list), ["product_id"])

In [0]:
L2_rules_df = spark.createDataFrame((L2_rules), ["X","Y","XY","conf"] )
L2_rules_df.show(3)

+--------+--------+------------------+-------------------+
|       X|       Y|                XY|               conf|
+--------+--------+------------------+-------------------+
|SNA40784|ELE17451|SNA40784->ELE17451| 0.1223529411764706|
|ELE91337|GRO46854|ELE91337->GRO46854|0.11404189294026378|
|ELE66600|FRO31317|ELE66600->FRO31317|0.07238762405137186|
+--------+--------+------------------+-------------------+
only showing top 3 rows



In [0]:
p_list_df.registerTempTable("p_list")
L2_rules_df.registerTempTable("l2_pairs")
l2_forconfcalculation.registerTempTable("l2_forconfcalculation")

#Create triplets from the productIDs found to be frequent in the second pass
query = """Select rrr.*, s2.S_XY as S_ZX
            from(
            select rr.*, s1.S_XY as S_YZ
            from(
            select r.*, s.S_XY as S_XY
            from(
            select l2.X as X, l2.Y as Y, p.product_id as Z
            from l2_pairs as l2 cross join p_list as p) as r
            join l2_forconfcalculation as s
            on r.X = s.X and r.Y = s.Y) as rr
            join l2_forconfcalculation as s1
            on rr.Y = s1.Y and rr.Z = s1.X) as rrr
            join l2_forconfcalculation as s2
            on rrr.Z = s2.Y and rrr.X = s2.X"""

c3 = spark.sql(query)
c3.show(5)

+--------+--------+--------+----+----+----+
|       X|       Y|       Z|S_XY|S_YZ|S_ZX|
+--------+--------+--------+----+----+----+
|ELE92920|ELE26917|DAI75645| 106| 278| 164|
|ELE92920|GRO73461|DAI75645| 108| 712| 164|
|ELE92920|SNA80324|DAI75645| 118|1130| 164|
|ELE92920|DAI85309|DAI75645| 201| 212| 164|
|ELE92920|GRO46854|DAI75645| 101| 190| 164|
+--------+--------+--------+----+----+----+
only showing top 5 rows



In [0]:
#This function will compute the support for the {X,Y,Z} itemsets.
def third_pass(row):
  
  p1 = row['X']
  p2 = row['Y']
  p3 = row['Z']
  browsing_len = len(browsing_history)
  count = 0

  if((p1 != p3) and (p2 != p3)):
    for i in range(browsing_len):
      if((p1 in browsing_history[i]) and (p2 in browsing_history[i]) and (p3 in browsing_history[i])):
        count = count+1
  else:
    count = 0

  return (p1, p2, p3, int(count), int(row['S_XY']), int(row['S_YZ']), int(row['S_ZX']))

In [0]:
c3_triples = c3.rdd.map(lambda row: third_pass(row))
l3 = c3_triples.filter(lambda x:x[3]>=100)
l3_rdd = l3.distinct()
l3_rdd.take(3)

[('ELE20847', 'DAI62779', 'DAI75645', 115, 275, 882, 306),
 ('ELE17451', 'DAI62779', 'DAI91290', 109, 1592, 353, 219),
 ('GRO71621', 'DAI62779', 'GRO73461', 153, 595, 1139, 305)]

In [0]:
def conf_cal_xyz(row):
  x = row[0]
  y = row[1]
  z = row[2]
  rule = x+" "+y+"->"+z
  conf = float(row[3])/float(row[4])
  return (x, y, z, rule,  float(conf))

In [0]:
def conf_cal_yzx(row):
  x = row[0]
  y = row[1]
  z = row[2]
  rule = y+" "+z+"->"+x
  conf = float(row[3])/float(row[5])
  return (y, z, x, rule,  float(conf))

In [0]:
def conf_cal_zxy(row):
  x = row[0]
  y = row[1]
  z = row[2]
  rule = z+" "+x+"->"+y
  conf = float(row[3])/float(row[6])
  return (z, x, y, rule,  float(conf))

In [0]:
#Calculate the confidence for all the association rules: {X,y -> Z}, {Y,Z -> X}, {Z,X -> Y}
l3_results_xyz = l3_rdd.map(lambda row: conf_cal_xyz(row))
l3_results_yzx = l3_rdd.map(lambda row: conf_cal_yzx(row))
l3_results_zxy = l3_rdd.map(lambda row: conf_cal_zxy(row))

l3_results = l3_results_xyz.union(l3_results_yzx)
l3_results = l3_results.union(l3_results_zxy)
l3_results = l3_results.distinct()
l3_results.sortBy(lambda r: -r[4]).take(5)

[('FRO53271', 'GRO85051', 'FRO40251', 'FRO53271 GRO85051->FRO40251', 1.0),
 ('GRO85051', 'SNA45677', 'FRO40251', 'GRO85051 SNA45677->FRO40251', 1.0),
 ('GRO85051', 'GRO21487', 'FRO40251', 'GRO85051 GRO21487->FRO40251', 1.0),
 ('ELE17451', 'GRO85051', 'FRO40251', 'ELE17451 GRO85051->FRO40251', 1.0),
 ('ELE20847', 'FRO92469', 'FRO40251', 'ELE20847 FRO92469->FRO40251', 1.0)]